### Import modules

In [1]:
import os
import pandas as pd

os.chdir("../../")

from datasets import load_dataset
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

In [2]:
from src.rag_pipeline import chunk_by_recursive_split, RAGSystem
from src.env_loader import load_api_keys
from src.ragas.ragas_pipeline import run_ragas_evaluation
from src import display_df

### Load API keys

In [3]:
openai_api_key = load_api_keys("OPENAI_API_KEY")

#### Initialize embeddings and RAG system

In [4]:
# embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings = OpenAIEmbeddings(api_key=openai_api_key, model='text-embedding-ada-002')

In [5]:
rag_system = RAGSystem(
  model_name = "gpt-4o",
  existing_vectorstore = False,
  embeddings = embeddings,
  clear_store = True
)

In [6]:
rag_system.initialize()

--Split 1000 documents into 5030 chunks.--


#### Test the RAG Chain

In [9]:
question = "What event is Rory McIlroy preparing for after the WGC-Cadillac Championship?"
result = rag_system.rag_chain.invoke(question)

In [10]:
result

{'question': 'What event is Rory McIlroy preparing for after the WGC-Cadillac Championship?',
 'answer': "Rory McIlroy is preparing for Arnold Palmer's tournament, which takes place from March 19-22.",
 'contexts': ['(CNN)Jordan Spieth has Rory McIlroy and the world No.1 spot firmly in his sights after winning the Valspar Championship on Sunday. Spieth won a three-way play-off with a 28-foot birdie on the third extra hole to become only the fourth player since 1940 to win twice on the PGA Tour before turning 22. It is a feat that not even McIlroy mastered with Tiger Woods, Sergio Garcia and Robert Gamez the only players to have achieved that particular accolade in the past 75 years. But it is the Northern Irishman that is within Spieth\'s focus heading towards Augusta. "I like studying the game, being a historian of the game," Spieth told the PGA Tour website. "It\'s really cool to have my name go alongside those. "But right now currently what I\'m really focused on is Rory McIlroy and

#### Initialize RAG system with ensemble_retriever with BM25 retriever

In [12]:
rag_system_ensemble = RAGSystem(
  model_name = "gpt-4o",
  existing_vectorstore = False,
  use_ensemble_retriever = True,
  embeddings=embeddings
)

In [13]:
rag_system_ensemble.initialize()

--Split 1000 documents into 5030 chunks.--


In [14]:
question = "What event is Rory McIlroy preparing for after the WGC-Cadillac Championship?"
result = rag_system_ensemble.rag_chain.invoke(question)

In [15]:
result

{'question': 'What event is Rory McIlroy preparing for after the WGC-Cadillac Championship?',
 'answer': 'Rory McIlroy is preparing for the U.S. Masters at Augusta after the WGC-Cadillac Championship.',
 'contexts': ['(CNN)Jordan Spieth has Rory McIlroy and the world No.1 spot firmly in his sights after winning the Valspar Championship on Sunday. Spieth won a three-way play-off with a 28-foot birdie on the third extra hole to become only the fourth player since 1940 to win twice on the PGA Tour before turning 22. It is a feat that not even McIlroy mastered with Tiger Woods, Sergio Garcia and Robert Gamez the only players to have achieved that particular accolade in the past 75 years. But it is the Northern Irishman that is within Spieth\'s focus heading towards Augusta. "I like studying the game, being a historian of the game," Spieth told the PGA Tour website. "It\'s really cool to have my name go alongside those. "But right now currently what I\'m really focused on is Rory McIlroy an

## RAGAS Pipeline testing the rag_chain

### Ragas Testing with Langsmith Tracing

In [7]:
experiment_name = "baseline_rag_benchmark_1"
dataset_name = "cnn_dailymail_evaluation"

rag_results = run_ragas_evaluation(
  rag_chain=rag_system.rag_chain,
  use_langsmith=True,
  experiment_name=experiment_name,
  dataset_name=dataset_name,
  upload_dataset_to_langsmith=True,
  save_results=True
)

--LOADING EVALUATION DATA--
--GETTING CONTEXT AND ANSWERS--
--USING LANGSMITH FOR EVALUATION--
Created a new dataset 'cnn_dailymail_evaluation'. Dataset is accessible at https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6
View the evaluation results for project 'baseline_rag_benchmark' at:
https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6/compare?selectedSessions=a58cdd46-9bf6-44ae-9ea4-f0853631205f

View all tests for Dataset cnn_dailymail_evaluation at:
https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6
[------------>                                     ] 5/19

Error evaluating run f591f3a5-4864-48c3-ac91-409ab305f428 with EvaluatorChain: APIConnectionError('Connection error.')
Traceback (most recent call last):
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/openai/_base_client.py", line 1558, in _request
    response = await self._client.send(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1661, in send
    response = await self._send_handling_auth(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1689, in _send_handling_auth
    response = await self._send_handling_redirects(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1726, in _send_handling_redirects
    response =

[------------------------------------------------->] 19/19

,feedback.answer_correctness,feedback.faithfulness,feedback.answer_relevancy,feedback.context_precision,error,execution_time,run_id
count,19.000000,18.000000,18.000000,18.000000,0,19.000000,19
unique,NaN,NaN,NaN,NaN,0,NaN,19
top,NaN,NaN,NaN,NaN,NaN,NaN,31f949c4-1476-4eb2-ae11-f23eb62af6d3
freq,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,0.706439,0.851852,0.887768,0.965509,NaN,2.434766,NaN
std,0.203250,0.243470,0.225174,0.083576,NaN,0.693174,NaN
min,0.229624,0.250000,0.000000,0.679167,NaN,1.334236,NaN
25%,0.579877,0.687500,0.918437,1.000000,NaN,2.051280,NaN
50%,0.743723,1.000000,0.934425,1.000000,NaN,2.481985,NaN
75%,0.832633,1.000000,0.963321,1.000000,NaN,2.726066,NaN


--EVALUATION COMPLETE--


AttributeError: 'TestResult' object has no attribute 'to_pandas'

### Run Ragas tests locally

In [32]:
rag_results = run_ragas_evaluation(
  rag_chain=rag_system.rag_chain,
  save_results=True,
  experiment_name="baseline_benchmark"
)

--LOADING EVALUATION DATA--
--EVALUATING LOCALLY--
--GETTING CONTEXT AND ANSWERS--


Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

--EVALUATION COMPLETE--
--RESULTS SAVED--


In [33]:
rag_results

,question,contexts,answer,ground_truth,answer_correctness,faithfulness,answer_relevancy,context_precision
0,What is the significance of cherry trees in Wa...,[Washington (CNN)There should be plenty of pen...,"The cherry trees in Washington, D.C., are sign...",Cherry trees hold great significance in Washin...,0.712666,0.833333,0.983714,1.000000
1,What is one of the events that Hillary Clinton...,"[(CNN)After a handful events in two months, Hi...",Hillary Clinton will be participating in a pai...,Hillary Clinton will be participating in the A...,0.998523,1.000000,0.944596,0.950000
2,What role did the United States-led coalition ...,[(CNN)In his first interview since Islamic Sta...,The United States-led coalition conducted airs...,The United States-led coalition played a role ...,0.618642,0.200000,0.938077,1.000000
3,What can viewers expect from the season finale...,[(CNN)Have Rick and his fellow survivors final...,Viewers can expect a 90-minute season finale f...,"The season finale of ""The Walking Dead"" is exp...",0.785931,1.000000,0.973125,1.000000
4,What is the significance of the Tunisian jihad...,"[Museum. Protesters held banners that said ""We...",The Tunisian jihadist's significance lies in h...,The significance of the Tunisian jihadist in r...,0.844000,1.000000,0.921761,1.000000
5,How did the undercover FBI informant play a ro...,[(CNN)An Army National Guard member and his co...,The undercover FBI informant played a crucial ...,The undercover FBI informant played a role in ...,0.902158,1.000000,0.852989,1.000000
6,How does the new Red brand aim to leverage Rad...,"[Carlson Rezidor, which is hueing (sic) toward...",The new Red brand aims to leverage Radisson's ...,The new Red brand aims to leverage Radisson's ...,0.548405,1.000000,0.915066,0.916667
7,When are the peak blooms expected during the N...,[Washington (CNN)There should be plenty of pen...,The peak blooms are expected between April 11 ...,The peak blooms are expected between April 11 ...,1.000000,1.000000,0.923510,1.000000
8,What is Zhanna Nemtsova's opinion on the Russi...,"[of the case except what is in the media. ""I d...",Zhanna Nemtsova has no faith in the Russian in...,Zhanna Nemtsova said she has no faith in the R...,0.534439,1.000000,0.000000,1.000000
9,Who was one of Putin's harshest critics?,[be heading an opposition party and do what I'...,One of Putin's harshest critics was Boris Nemt...,Boris Nemtsov,0.229628,1.000000,1.000000,1.000000
